# Autograd: 自动微分
在PyTorch的所有神经网络的核心就是<font color=red>autograd</font>包。让我们先简要地看看它，然后我们会训练我们的第一个神经网络。

<font color=red>autograd</font>包为张量上的所有操作提供自动微分。它是一个动态定义的框架。这意味着你的反向传播是由你的代码是如何运行而定义的，并且每一次迭代都可以不同。

让我们用一些例子来看看更多关于它的简单的术语。

## Tensor
torch.Tensor是包的核心类。如果将属性 <font color=red>.requires_grad</font> 设置为 <font color=red>True</font>，它就开始追踪跟其相关的所有操作。当你完成了你的计算时，可以调用 <font color=red>.backward()</font> 则所有的梯度计算将被自动完成。这个张量的梯度将被累加到 <font color=red>.grad</font> 属性中。

要停止一个tensor的追踪历史，你可以调用 <font color=red>.detach()</font> 方法来把它和计算历史分离开来，并且防止之后的计算不会被追踪。

为了防止追踪历史（和内存的使用），你可以用 <font color=red>with torch.no_grad()</font>: 来打包你的代码形成在 <font color=red>with torch.no_grad()</font>: 里面的代码块。这个方法在评估我们模型的时候显得尤为重要。因为模型可能拥有很多可训练的参数，且这些参数的 *requires_grad=True*，但是在评估阶段我们并不需要梯度。

这里还有一个类是对于 autograd 实现相当重要的，他就是 - a <font color=red>Function</font>.

<font color=red>Tensor</font> 和 <font color=red>Function</font> 相互内联从而建立了一个无环图。无环图编码了一个完整的计算历史。每个 tensor 都有个 <font color=red>.grad_fn</font> 的属性。这个属性能够推测出一个创建一个Tensor的 <font color=red>Function</font>（除非 Tensors 是被用户创建的，那么他们的 grad_fn 是 None）。

如果你想要计算微分，你可以在一个 Tensor 上调用 <font color=red>.backward()</font>。如果 Tensor 是个标量 （例如，它只是承载了一个元素数据）， 你不需要给定任何参数给 <font color=red>backward()</font> ，但是当 Tensor 有多个元素的数据时，你需要制定一个和该 tensor shape 对应的 tensor 作为梯度参数。

In [1]:
import torch

创建一个 tensor 然后把 require_grad 置为 True 来追踪和它相关的计算

In [2]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[ 1.,  1.],
        [ 1.,  1.]])


在 tensor 上面做个操作（计算）：

In [3]:
y = x + 2
print(y)

tensor([[ 3.,  3.],
        [ 3.,  3.]])


<font color=red>y</font>是有一个操作（加法）产生的结果，所以它拥有一个 <font color=red>grad_fn</font>.

In [4]:
print(y.grad_fn)

在y上面做更多的操作：

In [5]:
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[ 27.,  27.],
        [ 27.,  27.]]) tensor(27.)


<font color=red>.requires\_grad\_(...)</font> 会原位（in-place）改变 Tensors 的 requires_grad。当我们没有提供 requires_grad 的信息，默认的配置是 <font color=red>True</font>。

In [9]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


## 梯度
现在让我们来反向传播。因为 <font color=red>out</font> 只包含一个标量，所以 <font color=red>out.backward()</font> 和 <font color=red>out.backward( torch.tensor(1) )</font> 是等价的。

In [10]:
out.backward()

In [11]:
print(x.grad)

tensor([[ 4.5000,  4.5000],
        [ 4.5000,  4.5000]])


可以得到一个4.5的2x2矩阵。如果我们把 <font color=red>out</font> 这个 Tensor 称为 “o“ 的话。那么$$ o=\frac{1}{4}\sum_{i} z_i,z_i=3(x_i + 2)^2$$。所以我们知道o对于x中某个位置求偏导应该为$$\frac{3}{2}(x_i + 2)$$所以，偏导为9除以2等于4.5

你可以用 autograd 做很多疯狂的事！

In [12]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([  762.4423, -1363.4224,   -77.9226])


In [13]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(gradients)

print(x.grad)

tensor([  102.4000,  1024.0000,     0.1024])


你可以通过使用 <font color=red>.requires_grad=True by wrapping the code block in with torch.no_grad():</font> 停止 autograd 不在追踪 Tensors 的历史

In [19]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


## 后续阅读：
关于 <font color=red>autograd</font> 和 <font color=red>Function</font> 的文档位于https://pytorch.org/docs/stable/autograd.html